In [298]:
import copy


In [606]:
class csp_variable:
    """
    dom debe ser un set, para que ninguna tupla que representa el dominio de la variable se repita.
    var_id es el identificador de la variable. Por defecto es None.
    """
    def __init__(self, var_id = None, dom = set()):
        self.id = var_id
        self.dom = set(dom)
        self.variable = (var_id, dom)
    
    def __repr__(self):
        return "{id}".format(id=self.id)
    
    def Domain(self):
        return self.dom
    
    def getID(self):
        return self.id
    
    def modifyDomain(self, newDomain):
        self.dom = newDomain

In [814]:
class csp:
    def __init__(self):
        self.X = {} # k,v pairs where k is the variable identifier and v is the domain of k.
        self.D = self.getDomains() #k,v pair where k is X[i] and v is a set representing X[i]'s domain.
        self.C = [] #list of k,v tuples where k is the set of variables involved in the constraint, and v is the constraint.
                    # Every item in C is unique.
    
    def getArcs(self):
        self.updateDomains()
        return [Ci for Ci in self.C]
    
    '''
        Metodo que genera el grafo de restricciones del problema.
    '''
    def generateGraph(self):
        self.updateDomains()
        graph = dict()
        for Xi in self.X:
            # First we get the variable of which to get its neighbors
            #print('Selected Xi: {var}'.format(var = Xi))

            # We get the neighbors of Xi
            neighbors = list()

            # To get its neighbors we have to ask which variables are related to Xi in the constraints in C.
            #print('Constraints which involve Xi={0}:'.format(Xi))
            for constraint in self.C:
                if Xi in constraint:
                    constraint = set(constraint)
                    #print(constraint)
                    # For every constraint which involves Xi, we remove it and add it into a list of neighbors.
                    related_var = constraint - {Xi}
                    #print('The related variable to Xi={0} is {1}'.format(Xi, related_var))
                    # Once we get the related variable, we pop it off the related_var set to pass it as a regular object to a list
                    # To then have it hashed for uniqueness.
                    neighbors.append(related_var.pop())
                    graph[Xi] = neighbors
                
            #print('The neighbors of Xi={0} are: {1}'.format(Xi, neighbors))
        #print(graph)
        return graph
    
    '''
        Returns a copy of the list of neighbors of Xi
    '''
    def NeighborsOf(self, Xi):
        self.updateDomains()
        neighbors = copy.deepcopy(self.generateGraph()[Xi])
        return neighbors
    
    def updateDomains(self):
        self.D = [{Xi: Xi.Domain()} for Xi in self.X]
    
    def defineVariables(self, X):
        self.X = X
        self.updateDomains()
        
    def defineConstraints(self, C):
        self.C = C
        self.updateDomains()
        
    def getVariables(self):
        return self.X
        
    def getDomains(self):
        self.updateDomains()
        return self.D
        
    def getConstraints(self):
        self.updateDomains()
        return self.C
    
    def __repr__(self):
        self.updateDomains()
        return "CSP: \n X ={vars},\n D ={domains},\n C ={constraints}\n".format(
            vars = self.X, 
            domains=self.D, 
            constraints=self.C)
    

In [380]:
""" CSP class testing grounds """

problem = csp()

x = csp_variable('x', (1, 2, 3))
y = csp_variable('y', (1, 2, 3))
z = csp_variable('z', (1, 2, 3))

X = {
    x.getID():x.Domain(),
    y.getID():y.Domain(),
    z.getID():z.Domain()
}
print(X)

D = {
    x.getID():x.Domain(),
    y.getID():y.Domain(),
    z.getID():z.Domain()
}

print(D)

C1 = (
        ('x', 'y', 'z'), 
        {(xi, yj, zk) for xi in X['x'] for yj in X['y'] for zk in X['z'] if xi + yj == zk }
    )

C2 = (
        ('x', 'y'), 
        {(xi, yj) for xi in X['x'] for yj in X['y'] if xi < yj}
    )

C = [C1, C2]

print(C)

problem.X = X
problem.D = D
problem.C = C

print(problem.getArcs())

{'x': {1, 2, 3}, 'y': {1, 2, 3}, 'z': {1, 2, 3}}
{'x': {1, 2, 3}, 'y': {1, 2, 3}, 'z': {1, 2, 3}}
[(('x', 'y', 'z'), {(2, 1, 3), (1, 2, 3), (1, 1, 2)}), (('x', 'y'), {(1, 2), (1, 3), (2, 3)})]
[('x', 'y', 'z'), ('x', 'y')]


In [151]:
""" csp_variables testing grounds"""

x = csp_variable('x', set((1,2,3)))

print(x.Domain())


{1, 2, 3}
a
 b
 c


In [565]:
""" Australia CSP definition testing grounds """

problem = csp()

Di = set(['red', 'green', 'blue'])

WA = csp_variable('WA', Di)
NT = csp_variable('NT', Di)
Q = csp_variable('Q', Di)
NSW = csp_variable('NSW', Di)
V = csp_variable('V', Di)
SA = csp_variable('SA', Di)
T = csp_variable('T', Di)

problem.X = set([WA, NT, Q, NSW, V, SA, T])
problem.D = [Xi.Domain() for Xi in problem.X]
problem.C = {
    (WA, SA): {(vi, vj) for vi in WA.Domain() for vj in SA.Domain() if vi is not vj},
    (NT, SA): {(vi, vj) for vi in NT.Domain() for vj in SA.Domain() if vi is not vj},
    (SA, Q): {(vi, vj) for vi in SA.Domain() for vj in Q.Domain() if vi is not vj},
    (NSW, SA): {(vi, vj) for vi in NSW.Domain() for vj in SA.Domain() if vi is not vj},
    (V, SA): {(vi, vj) for vi in V.Domain() for vj in SA.Domain() if vi is not vj},
    (WA, NT): {(vi, vj) for vi in WA.Domain() for vj in NT.Domain() if vi is not vj},
    (NT, Q): {(vi, vj) for vi in NT.Domain() for vj in Q.Domain() if vi is not vj},
    (Q, NSW): {(vi, vj) for vi in Q.Domain() for vj in NSW.Domain() if vi is not vj},
    (NSW, V): {(vi, vj) for vi in NSW.Domain() for vj in V.Domain() if vi is not vj}
}

In [483]:
""" Generate graph testing grounds """

"""
X = {Xi, Xj, Xk}
C = { 
        (Xi, Xj): {Xi != Xj},
        (Xj, Xk): {Xj != Xk}
   } 
    graph = {
        Xi: {Xj},
        Xj: {Xi, Xk},
        Xk: {Xj}
    }

print(problem.C[(WA, SA)])
self = problem
arcs = {(arc) for arc in self.getArcs()}
graph = {}
for Xi in self.X:
    neighbors = set()
    for arc in arcs:
        if Xi in arc:
            #print(Xi,set(arc))
            neighbors.add(set(arc).remove(Xi))
    
    graph[Xi] = neighbors

print (arcs)
print(graph)
"""

# First we get the variable of which to get its neighbors
Xi = problem.X.pop()
print('Selected Xi: {var}'.format(var = Xi))

# We get the neighbors of Xi
neighbors = list()
graph = dict()

# To get its neighbors we have to ask which variables are related to Xi in the constraints in C.
print('Constraints which involve Xi={0}:'.format(Xi))
for constraint in problem.C:
    if Xi in constraint:
        constraint = set(constraint)
        print(constraint)
        # For every constraint which involves Xi, we remove it and add it into a list of neighbors.
        related_var = constraint - {Xi}
        print('The related variable to Xi={0} is {1}'.format(Xi, related_var))
        # Once we get the related variable, we pop it off the related_var set to pass it as a regular object to a list
        # To then have it hashed for uniqueness.
        neighbors.append(related_var.pop())
        graph[Xi] = neighbors
        
print('The neighbors of Xi={0} are: {1}'.format(Xi, neighbors))
print(graph)

Selected Xi: SA
Constraints which involve Xi=SA:
{WA, SA}
The related variable to Xi=SA is {WA}
{NT, SA}
The related variable to Xi=SA is {NT}
{Q, SA}
The related variable to Xi=SA is {Q}
{NSW, SA}
The related variable to Xi=SA is {NSW}
{V, SA}
The related variable to Xi=SA is {V}
The neighbors of Xi=SA are: [WA, NT, Q, NSW, V]
{SA: [WA, NT, Q, NSW, V]}


In [487]:
graph = dict()
for Xi in problem.X:
    # First we get the variable of which to get its neighbors
    print('Selected Xi: {var}'.format(var = Xi))

    # We get the neighbors of Xi
    neighbors = list()

    # To get its neighbors we have to ask which variables are related to Xi in the constraints in C.
    print('Constraints which involve Xi={0}:'.format(Xi))
    for constraint in problem.C:
        if Xi in constraint:
            constraint = set(constraint)
            print(constraint)
            # For every constraint which involves Xi, we remove it and add it into a list of neighbors.
            related_var = constraint - {Xi}
            print('The related variable to Xi={0} is {1}'.format(Xi, related_var))
            # Once we get the related variable, we pop it off the related_var set to pass it as a regular object to a list
            # To then have it then hashed for uniqueness.
            neighbors.append(related_var.pop())
            graph[Xi] = neighbors
        
    print('The neighbors of Xi={0} are: {1}'.format(Xi, neighbors))
print(graph)

Selected Xi: V
Constraints which involve Xi=V:
{V, SA}
The related variable to Xi=V is {SA}
{V, NSW}
The related variable to Xi=V is {NSW}
The neighbors of Xi=V are: [SA, NSW]
Selected Xi: Q
Constraints which involve Xi=Q:
{SA, Q}
The related variable to Xi=Q is {SA}
{Q, NT}
The related variable to Xi=Q is {NT}
{Q, NSW}
The related variable to Xi=Q is {NSW}
The neighbors of Xi=Q are: [SA, NT, NSW]
Selected Xi: T
Constraints which involve Xi=T:
The neighbors of Xi=T are: []
Selected Xi: NT
Constraints which involve Xi=NT:
{SA, NT}
The related variable to Xi=NT is {SA}
{NT, WA}
The related variable to Xi=NT is {WA}
{Q, NT}
The related variable to Xi=NT is {Q}
The neighbors of Xi=NT are: [SA, WA, Q]
Selected Xi: WA
Constraints which involve Xi=WA:
{SA, WA}
The related variable to Xi=WA is {SA}
{NT, WA}
The related variable to Xi=WA is {NT}
The neighbors of Xi=WA are: [SA, NT]
Selected Xi: SA
Constraints which involve Xi=SA:
{SA, WA}
The related variable to Xi=SA is {WA}
{SA, NT}
The relat

In [833]:
"""
def Revise(csp, Xi, Xj):
    revised = False
    Di = Xi.Domain()
    Dj = Xj.Domain()
    
    # for each vk in Di
    # if no value vm in Dj allows (vk, vm) to satisfy the constraint between Xi and Xj then
    # remove vk from Di and set revised to true.
    
    # In order to accquire for vk the value vm which violates the constraint we remove the
    diff = [(vk ,Dj - {vk}) for vk in Di]
    print(diff)
    for tupl in diff:
        if len(tupl[1]) == 0:
            Di.remove(tupl[0])        
            
            revised = True
    return revised
"""

def Revise(csp, Xi, Xj):
    revised = False
    for vk in Xi.Domain():
        # Let's check in which order the key is in:
        #print('view of csp\'s constraints: {0}'.format(csp.C.keys()))
        if (Xi, Xj) in csp.C.keys():
            #print('({0},{1}) is a key of the constraint dict'.format(Xi, Xj))
            constraint = (Xi, Xj)
        elif (Xj, Xi) in csp.C.keys():
            #print('({0},{1}) is a key of the constraint dict'.format(Xj, Xi))
            constraint = (Xj, Xi)
    
        if not any((vk, vm) in csp.C[constraint] for vm in Xj.Domain()):
            Xi.modifyDomain(Xi.Domain() - {vk})
            
    revised = True
    

In [827]:
""" Revise testing grounds """

test_problem = csp()

a = csp_variable('a', {1, 2, 3})
b = csp_variable('b', {1, 2, 3})

test_problem.X = {a, b}
test_problem.D = [Xi.Domain() for Xi in test_problem.X]
test_problem.C = {
    (a, b):{(vk, vm) for vk in a.Domain() for vm in b.Domain() if vk+vm == 3}
}

print(test_problem.C[a, b])


print(
    Revise(test_problem, a, b)
)

print(test_problem.NeighborsOf(a))

{(1, 2), (2, 1)}
view of csp's constraints: dict_keys([(a, b)])
(a,b) is a key of the constraint dict
Pair found!
view of csp's constraints: dict_keys([(a, b)])
(a,b) is a key of the constraint dict
Pair found!
view of csp's constraints: dict_keys([(a, b)])
(a,b) is a key of the constraint dict
Not found in constraint.
None
[b]


In [834]:
def AC3(csp):
    queue = csp.getArcs()
    graph = csp.generateGraph()
    
    while len(queue) != 0:
        (Xi, Xj) = queue.pop()
        #print('Constraint being evaluated: ({0},{1})'.format(Xi, Xj))
        if Revise(csp, Xi, Xj):
            #print('{0}\'s domain: {1}'.format(Xi, Xi.Domain()))
            #print('{0}\'s domain: {1}'.format(Xj, Xj.Domain()))
            if len(Xi.Domain()) == 0:
                return False
            #print('Xi={0}\'s neighbors: {1}'.format(Xi, csp.NeighborsOf(Xi)))
            #print('Xj={0}'.format(Xj))
            
            neighbors_without_Xj = set(csp.NeighborsOf(Xi)) - {Xj}
            if len(neighbors_without_Xj) == 0:
                break
            
            for Xk in neighbors_without_Xj:
                queue.append((Xk, Xi))

    return True

In [839]:
""" AC3 testing grounds """

test_problem = csp()

Di = set(['red', 'green', 'blue'])

WA = csp_variable('WA', Di)
NT = csp_variable('NT', Di)
Q = csp_variable('Q', Di)
NSW = csp_variable('NSW', Di)
V = csp_variable('V', Di)
SA = csp_variable('SA', Di)
T = csp_variable('T', Di)

test_problem.X = set([WA, NT, Q, NSW, V, SA, T])
test_problem.D = [Xi.Domain() for Xi in test_problem.X]
test_problem.C = {
    (WA, SA): {(vi, vj) for vi in WA.Domain() for vj in SA.Domain() if vi is not vj},
    (NT, SA): {(vi, vj) for vi in NT.Domain() for vj in SA.Domain() if vi is not vj},
    (SA, Q): {(vi, vj) for vi in SA.Domain() for vj in Q.Domain() if vi is not vj},
    (NSW, SA): {(vi, vj) for vi in NSW.Domain() for vj in SA.Domain() if vi is not vj},
    (V, SA): {(vi, vj) for vi in V.Domain() for vj in SA.Domain() if vi is not vj},
    (WA, NT): {(vi, vj) for vi in WA.Domain() for vj in NT.Domain() if vi is not vj},
    (NT, Q): {(vi, vj) for vi in NT.Domain() for vj in Q.Domain() if vi is not vj},
    (Q, NSW): {(vi, vj) for vi in Q.Domain() for vj in NSW.Domain() if vi is not vj},
    (NSW, V): {(vi, vj) for vi in NSW.Domain() for vj in V.Domain() if vi is not vj}
}

"""
a = csp_variable('a', {1, 2, 3})
b = csp_variable('b', {1, 2, 3})

test_problem.X = {a, b}
test_problem.D = [Xi.Domain() for Xi in test_problem.X]
test_problem.C = {
    (a, b):{(vk, vm) for vk in a.Domain() for vm in b.Domain() if vk+vm == 3}
}
"""

"""
a = csp_variable('a', {1, 2, 3})
b = csp_variable('b', {1, 2, 3})
c = csp_variable('c', {1, 2, 3})

test_problem.X = {a, b, c}
test_problem.D = [Xi.Domain() for Xi in test_problem.X]
test_problem.C = {
    (a, b):{(vk, vm) for vk in a.Domain() for vm in b.Domain() if vk+vm == 3},
    (b, c): {(vk, vm) for vk in b.Domain() for vm in c.Domain() if vk>vm},
    (a, c):{(vk, vm) for vk in a.Domain() for vm in c.Domain() if vk != vm}
}
"""

print(test_problem)

AC3(test_problem)

print(test_problem)

CSP: 
 X ={SA, V, Q, NT, WA, T, NSW},
 D =[{SA: {'green', 'blue', 'red'}}, {V: {'green', 'blue', 'red'}}, {Q: {'green', 'blue', 'red'}}, {NT: {'green', 'blue', 'red'}}, {WA: {'green', 'blue', 'red'}}, {T: {'green', 'blue', 'red'}}, {NSW: {'green', 'blue', 'red'}}],
 C ={(WA, SA): {('green', 'blue'), ('red', 'blue'), ('blue', 'green'), ('red', 'green'), ('blue', 'red'), ('green', 'red')}, (NT, SA): {('green', 'blue'), ('red', 'blue'), ('blue', 'green'), ('red', 'green'), ('blue', 'red'), ('green', 'red')}, (SA, Q): {('green', 'blue'), ('red', 'blue'), ('blue', 'green'), ('red', 'green'), ('blue', 'red'), ('green', 'red')}, (NSW, SA): {('green', 'blue'), ('red', 'blue'), ('blue', 'green'), ('red', 'green'), ('blue', 'red'), ('green', 'red')}, (V, SA): {('green', 'blue'), ('red', 'blue'), ('blue', 'green'), ('red', 'green'), ('blue', 'red'), ('green', 'red')}, (WA, NT): {('green', 'blue'), ('red', 'blue'), ('blue', 'green'), ('red', 'green'), ('blue', 'red'), ('green', 'red')}, (NT, Q): {

In [4]:
class value:
    def __init__(self):
        pass
    
    def IsConsistent(self, assignment):
        pass

In [ ]:
def Select_Unasigned_Variable(csp):
    pass

In [ ]:
def Order_Domain_Values(var, assignment, csp):
    pass

In [ ]:
def Select_Unasigned_Variable(csp):
    pass

In [ ]:
def Backtrack(assignment, csp):
    failure = none
    
    if complete(assignment):
        return assignment
    
    var = Select_Unasigned_Variable(csp)
    
    for value in Order_Domain_Values(var, assignment, csp):
        if value.IsConsistent(assignment):
            assignment.add(value)
            inferences = Inference(csp, var, value)
            if inferences.IsNotFailure():
                assignment.add(inferences)
                result = Backtrack(assignment, csp)
                if result.IsNotFailure():
                    return result
        assignment.remove(value)
        assignment.remove(inferences)
    return failure
            

In [3]:
def Backtracking_Search(csp)
    return Backtrack([], csp)

set()
